# Chebyshev Iteration Method


## Theory
### Chebyshev polynomials
* Recurrence definition
$$
\begin{aligned}
    T_0(x) &= 1 \\
    T_1(x) &= x \\
    T_{n+1}(x) &= 2x T_n(x) - T_{n-1}(x) 
\end{aligned}
$$

* Trigonometric definition
$$
    T_n(x) = \begin{cases}
        \cos (n\operatorname{arcos} x) &\qquad \text{if } |x| \le 1 \\
        \cosh (n\operatorname{arcosh} x) &\qquad \text{if } x \ge 1 \\
        (-1)^n \cosh (n\operatorname{arcosh}(-x)) &\qquad \text{if } x \le -1
    \end{cases}
$$

* Minimal $\infty$-norm [1]

For any given $n \ge 1$, among the polynomials of degree $n$ with leading coefficient 1 (monic polynomials),
$$
    f(x) = \frac{1}{2^{n-1}} T_n(x)
$$
is the one of which the maximal absolute value on the interval $[-1, 1]$ is minimal.

### Iterative Solver
Solving linear system
$$
    A x = b
$$
where $A$ is a positive definite matrix. 

Spectrum of $A$: $\mathrm{spectrum}(A) = \{\lambda_1, \dotsc, \lambda_n \}$, where $0 < \lambda_1 \le \dotsb \le \lambda_n$.

* General formulation of Iterative solver
$$
    x_{k+1} = x_k + \alpha_{k+1} (b - A x_k)
$$

Let error $e_{k} = x - x_k$
$$
    \begin{aligned}
        e_{k+1} &= x - x_{k+1} \\
        &= (x - x_k) - (x_{k+1} - x_k) \\
        &= (x - x_k) -\alpha_{k+1} (b - A x_k) \\
        &= e_{k} - \alpha_{k+1} A(x - x_k) \\
        &= (I - \alpha_{k+1} A) e_k
    \end{aligned}
$$
Then,
$$
    e_{k} = \prod_{i=1}^{k} (I - \alpha_k A) e_0
$$

* Norm of error:
$$
    \| e_k \| = \prod_{i=1}^{k} \| (I - \alpha_k A) e_0 \| \le \prod_{i=1}^{k} (\| I - \alpha_k A \|) \cdot \| e_0 \|
$$

To minimize the error after $k$ iteration $\Rightarrow$ minimize $\prod_{i=1}^{k} \| I - \alpha_k A \|$.

### Richardson method:
$\alpha_k \equiv \alpha$.
$$
    x_{k+1} = x_k + \alpha (b - A x_k)
$$

In this case, minimize $\prod_{i=1}^{k} \| I - \alpha_k A \| \Rightarrow$ minimize $ \| I - \alpha A \| \Rightarrow$ minimize $ \max_{\lambda \in \mathrm{spectrum}(A)} |(1 - \alpha \lambda)|$.

Choosing the best $\alpha$:
$$
    \alpha^* = \operatorname*{argmin}_{\alpha} \max_{\lambda \in \mathrm{spectrum}(A)} |1 - \alpha \lambda|
$$
$$
    1 - \alpha^* \lambda_1 = \alpha^* \lambda_n - 1  \Rightarrow \alpha^* = \frac{2}{\lambda_1 + \lambda_n}
$$

### Chebyshev iteration [5]
Choosing the best $\alpha_i, i=1, 2, \dotsc, k$.
$$
    \{\alpha_i\}_{i=1,\dotsc,k} = \operatorname*{argmin}_{\alpha_i} \left\{ \max_{\lambda \in \mathrm{spectrum}(A)} |(1-\alpha_1 \lambda) \dotsb (1-\alpha_k \lambda)| \right\}
$$
Suppose that we don't know all eigenvalues of $A$ and only know $M = \lambda_n$ and $m = \lambda_1$.

Relaxation: $\lambda \in \mathrm{spectrum}(A) \Rightarrow \lambda \in [m, M]$.
$$
    \{\alpha_i\}_{i=1,\dotsc,k} = \operatorname*{argmin}_{\alpha_i} \left\{ \max_{\lambda \in [m, M]} |(1-\alpha_1 \lambda) \dotsb (1-\alpha_k \lambda)| \right\}
$$

Find a degree-$k$ polynomial $p_k(x)$ with $p_k(0) = 1$, which minimizes $\max_{\lambda \in [m, M]} | p_k(\lambda) |$ $\Rightarrow$ The minimal $\infty$-norm property of Chebyshev polynomial.  

Parameterize $\lambda \in [m, M]$ as $\lambda = \frac{M+m}{2} + \frac{M-m}{2} t$ and $t \in [-1, 1]$.

Optimal polynomial:
$$
    p_k(\lambda) = T_k\left( \frac{\lambda - (M+m)/2}{(M-m)/2} \right) \bigg/ T_k \left( -\frac{M+m}{M-m} \right) = \prod_{i=1}^{k} (1-\alpha_i t)
$$

Corresponing $\alpha_k$ (zero-points of Chebyshev polynomial):
$$
    \alpha_i^{-1} = \frac{1}{2} \left( M + m + (M - m)\cos \frac{\pi(2i-1)}{2k} \right) \quad i=1,2,\dotsc,k
$$

### Three-term Chebyshev iteration [3, 4]
Practical version in computation (three-term Chebyshev iteration):
$$
    x_{k+1} = x_{k} + \alpha_{k+1}(r - Ax_k) + \beta_{k+1}(x_k - x_{k-1})
$$
where
$$
    \begin{aligned}
        \alpha_{k+1} &= \begin{cases}
            1 / d & \quad k = -1 \\
            2d / (2d^2 - c^2) & \quad k = 0 \\
            (d - (c/2)^2 \alpha_{k-1})^{-1} & \quad k \ge 1 
        \end{cases} \\
        \beta_{k+1} &= \begin{cases}
            0 & \quad k = -1 \\
            d \cdot \alpha_{k+1} - 1 & \quad k \ge 0
        \end{cases}
    \end{aligned}
$$
and $d = (\lambda_{max} + \lambda_{min}) / 2, c = (\lambda_{max} - \lambda_{min}) / 2$.



## Algorithm & Code [2]
算法：

![alg](./chebyshev_alg.png)

In [1]:
import numpy as np

The code below seems different from the definitions and iteration rules of $\alpha_k$ and $\beta_k$ above.

But it can be proved that they generate the same $\alpha_k$ and $\beta_k$ sequences (mathematical induction).

In [2]:
def chebyshev_solve(x0, A, b, lmin, lmax, max_iter=100, epsilon=1-10):
    d = (lmax + lmin) / 2
    c = (lmax - lmin) / 2
    x = x0
    r = b - A @ x

    for i in range(max_iter):
        z = r
        if i == 0:
            p = z
            alpha = 1 / d
        elif i == 1:
            beta = 1/2 * (c * alpha) ** 2
            alpha = 1 / (d - beta / alpha)
            p = z + beta * p
        else:
            beta = (c * alpha / 2) ** 2
            alpha = 1 / (d - beta / alpha)
            p = z + beta * p
    
        x = x + alpha * p
        r = b - A @ x

        if np.linalg.norm(r) < epsilon:
            break
    return x

In [3]:
np.random.seed(42)
N = 10

A = np.random.randn(N, N)
A = A @ A.T
eigvals = np.linalg.eigvals(A)
lmax, lmin = eigvals[0], eigvals[-1]
print("kappa: ", lmax / lmin)
x_star = np.random.randn(N)

b = A @ x_star

x = chebyshev_solve(np.zeros_like(x_star), A, b, lmin, lmax, max_iter=1000)
print(np.linalg.norm(x - x_star))


kappa:  56.84915573303675
2.8596922373412013e-08


**Remark**

Note that Chebyshev iteration is designed for large sparse linear systems. Above code uses a dense matrix $A$ since it is only an example.

## References
1. https://en.wikipedia.org/wiki/Chebyshev_polynomials
2. https://en.wikipedia.org/wiki/Chebyshev_iteration
3. Elman H C, Saad Y, Saylor P E. A hybrid Chebyshev Krylov subspace algorithm for solving nonsymmetric systems of linear equations[J]. SIAM Journal on Scientific and Statistical Computing, 1986, 7(3): 840-855.
4. Manteuffel T A. The Tchebychev iteration for nonsymmetric linear systems[J]. Numerische Mathematik, 1977, 28(3): 307-327.
5. Iterative methods for linear systems: theory and applications[M]. Society for Industrial and Applied Mathematics, 2014.
